# Tensorflow Object Detection with custom dataset in Google Colab

Jupyter notebook providing steps to retrain a ModelZoo model with custom dataset.

It runs in Google Colab using Tensorflow Object Detection API.

**Requirements are only dataset images and annotations file.**

The code is compatible with Object Detection api updates in may 13 2019 release.

Colab Runtime type: Python2, GPU enabled.

# Create Dataset

I generated dataset annotations with LabelImg.

Notebook train a model for one class object detection. It is possible to slightly modify notebook to train model for multiple classes.

Before running notebook, we need to create dataset:


1.   Collect various pictures of objects to detect
2.   Rename image filenames with format objectclass_seq.jpg
3.   In LabelImg create annotation files. LabelImg saves annotations as XML files in PASCAL VOC format
4.   Create dataset.zip file having structure defined below
5.  Upload the zip file in your Google Drive

Zip file structure:



*   dataset.zip file
*   |-images directory
*       ->|-image files (filename format:objectclass_seq.jpg)
*   |-annotations directory
*      ->|-xmls directory
*     ->|-annotation files (filename format:objectclass_seq.xml)    
      

Where objectclass is the class name, seq is a sequence number (001, 002, 003, ...)

Check my dataset.zip file as dataset example.



# Install required packages
Just the things that are missing

In [1]:
%cd ~

!rm ~/models -R
  
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip install -q pycocotools

%cd ~/models/research
!protoc object_detection/protos/*.proto --python_out=.

/root
rm: cannot remove '/root/models': No such file or directory
     |████████████████████████████████| 993kB 3.9MB/s 
/root/models/research


# Install tensorflow >1.4

In [0]:
#!pip uninstall tensorflow -y
#!pip uninstall tensorflow-estimator -y
#!pip install tensorflow-gpu==1.12
#!pip install tensorflow==1.12
#!pip install tensorflow_hub

# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [3]:

import tensorflow as tf
tf.__version__
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


# Testing environmet

In [4]:
%cd ~/models/research
!mkdir train eval
import os
os.environ ['PYTHONPATH'] = "~/models/research"
os.environ['PYTHONPATH'] += ":~/models/research/slim"
!echo $PYTHONPATH

%cd ~/models/research
!protoc object_detection/protos/*.proto --python_out=.
%cd ~/models/research
!python object_detection/builders/model_builder_test.py

/root/models/research
~/models/research:~/models/research/slim
/root/models/research
/root/models/research
Traceback (most recent call last):
  File "object_detection/builders/model_builder_test.py", line 23, in <module>
    from object_detection.builders import model_builder
ModuleNotFoundError: No module named 'object_detection'


In [5]:
%ls


a3c_blogpost/                      lm_commonsense/
adversarial_crypto/                lstm_object_detection/
adversarial_logit_pairing/         marco/
adversarial_text/                  maskgan/
adv_imagenet_models/               minigo/
astronet/                          morph_net/
attention_ocr/                     namignizer/
audioset/                          neural_gpu/
autoaugment/                       neural_programmer/
autoencoder/                       next_frame_prediction/
brain_coder/                       nst_blogpost/
cognitive_mapping_and_planning/    object_detection/
cognitive_planning/                pcl_rl/
compression/                       ptn/
cvt_text/                          qa_kg/
deep_contextual_bandits/           README.md
deeplab/                           real_nvp/
deep_speech/                       rebar/
delf/                              resnet/
differential_privacy/              sentiment_analysis/
domain_adaptation/                 seq2species/
effic

In [6]:
!python setup.py build
!python setup.py install

running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/inputs.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection/export_inference_graph.py -> build/lib/object_detection
copying object_detection/model_main.py -> build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detection/model_hparams.py -> build/lib/object_detection
copying object_detection/model_lib_test.py -> build/lib/object_detection
copying object_detection/model_tpu_main.py -> build/lib/object_detection
copying object_detection/__init__.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib.py -> build/lib/object_detection
copying object_detection/model_lib.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_test.py -> build/lib/object_detection
copying object

In [7]:
%cd slim

/root/models/research/slim


In [8]:
!pip install -e .

Obtaining file:///root/models/research/slim
  Running setup.py develop for slim


In [9]:
!pip install -U pandas
!pip install -U matplotlib

Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (0.24.2)
Requirement already up-to-date: matplotlib in /usr/local/lib/python3.6/dist-packages (3.0.3)


In [10]:
%cd ..
!python object_detection/builders/model_builder_test.py

/root/models/research

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.055s

OK (skipped=1)


In [11]:
%cd ~
!mkdir datalab

/root


In [12]:
#@title
%cd ~/datalab

/root/datalab


# Download and extract dataset



*  Change name attribute in label_map, accordingly with objectclass filename.

*   Substitute fileId value with your dataset.zip id in Google Drive. See here my answer to get file id.


# Empty png files

Create empty png mask files to avoid error in create_pet_tf_record.py, they are not used in training model.

In [13]:
!echo "item { id: 1 name: 'person'}" > label_map.pbtxt


fileId = '1Bi4pS57TXVSO4xK4H3sEa6uJ6SW9-yk5'

import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileName = fileId + '.zip'
downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile(fileName)
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)

image_files=os.listdir('images')
im_files=[x.split('.')[0] for x in image_files]
with open('annotations/trainval.txt', 'w') as text_file:
  for row in im_files:
    text_file.write(row + '\n')

Extracted zip file 1Bi4pS57TXVSO4xK4H3sEa6uJ6SW9-yk5.zip


In [14]:
%cd ~/datalab/annotations

!mkdir trimaps

from PIL import Image
image = Image.new('RGB', (640, 400))

for filename in os.listdir('xmls'):
  filename = os.path.splitext(filename)[0]
  image.save('trimaps/' + filename + '.png')

/root/datalab/annotations


In [15]:
!ls

trainval.txt  trimaps  xmls


In [16]:
!apt-get -y install protobuf-compiler python-pil python-lxml

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 16 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,688 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4

# Create TFRecord

In [17]:
%cd ~/datalab

!python ~/models/research/object_detection/dataset_tools/create_pet_tf_record.py --label_map_path=label_map.pbtxt --data_dir=. --output_dir=. --num_shards=1



/root/datalab
/usr/local/lib/python3.6/dist-packages/object_detection-0.1-py3.6.egg/object_detection/utils/dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:


In [18]:
!ls

annotations	 pet_faces_train.record-00000-of-00001
images		 pet_faces_val.record-00000-of-00001
label_map.pbtxt


In [0]:
!mv pet_faces_train.record-00000-of-00001 tf_train.record

!mv pet_faces_val.record-00000-of-00001 tf_val.record

In [20]:
!ls

annotations  images  label_map.pbtxt  tf_train.record  tf_val.record


# Download pretrained model

Cell downloads **ssd_mobilenet_v1_coco** model to use as starting checkpoint.

To use another model from ModelZoo change MODEL var.

In [21]:
!pip install pytest-shutil

  Stored in directory: /root/.cache/pip/wheels/61/61/c7/5f031e2fbd95084d1b4f31f2e959abed71059a11abafe10d4f
Successfully built pytest-shutil


In [22]:

%cd ~/datalab

fileId = '1zqBNKF9hhhuoRVgHnavvT7BnF50vkFH5'
MODEL=fileId
import os
import shutil 
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import tarfile
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileName = fileId + '.tar.gz'
downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile(fileName)


DEST_DIR = 'pretrained_model1'
MODEL_FILE = MODEL + '.tar.gz'
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename('ssd_mobilenet_v1_coco_2017_11_17', DEST_DIR)


/root/datalab


In [23]:
!ls -l

total 170088
drwxr-xr-x 4 root   root      4096 May 14 19:27 annotations
drwxr-xr-x 2 root   root     36864 May 14 19:27 images
-rw-r--r-- 1 root   root        29 May 14 19:26 label_map.pbtxt
drwxr-x--- 3 275875 5000      4096 Nov 18  2017 pretrained_model1
-rw-r--r-- 1 root   root 121490923 May 14 19:27 tf_train.record
-rw-r--r-- 1 root   root  52627123 May 14 19:27 tf_val.record


In [24]:
%cd pretrained_model1
!ls
%cd ..

/root/datalab/pretrained_model1
checkpoint		   model.ckpt.data-00000-of-00001  model.ckpt.meta
frozen_inference_graph.pb  model.ckpt.index		   saved_model
/root/datalab


In [25]:
%cd ~/datalab

#import os
#import shutil
#import glob
#import urllib
#import tarfile

#MODEL = 'ssd_mobilenet_v1_coco_2017_11_17'
#MODEL_FILE = MODEL + '.tar.gz'
#DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
#DEST_DIR = 'pretrained_model1'

#if not (os.path.exists(MODEL_FILE)):
 # opener = urllib.URLopener()
  #opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#tar = tarfile.open(MODEL_FILE)
#tar.extractall()
#tar.close()
#
#os.remove(MODEL_FILE)
#if (os.path.exists(DEST_DIR)):
#  shutil.rmtree(DEST_DIR)
#os.rename(MODEL, DEST_DIR)

/root/datalab



# Edit model config file

To you use a different pretrained model in step before, update accordingly filename var and re.sub functions in nex

In [26]:
%cd ~/models/research/object_detection/samples/configs/
!ls

/root/models/research/object_detection/samples/configs
embedded_ssd_mobilenet_v1_coco.config
facessd_mobilenet_v2_quantized_320x320_open_image_v4.config
faster_rcnn_inception_resnet_v2_atrous_coco.config
faster_rcnn_inception_resnet_v2_atrous_cosine_lr_coco.config
faster_rcnn_inception_resnet_v2_atrous_oid.config
faster_rcnn_inception_resnet_v2_atrous_oid_v4.config
faster_rcnn_inception_resnet_v2_atrous_pets.config
faster_rcnn_inception_v2_coco.config
faster_rcnn_inception_v2_pets.config
faster_rcnn_nas_coco.config
faster_rcnn_resnet101_atrous_coco.config
faster_rcnn_resnet101_ava_v2.1.config
faster_rcnn_resnet101_coco.config
faster_rcnn_resnet101_fgvc.config
faster_rcnn_resnet101_kitti.config
faster_rcnn_resnet101_pets.config
faster_rcnn_resnet101_voc07.config
faster_rcnn_resnet152_coco.config
faster_rcnn_resnet152_pets.config
faster_rcnn_resnet50_coco.config
faster_rcnn_resnet50_fgvc.config
faster_rcnn_resnet50_pets.config
mask_rcnn_inception_resnet_v2_atrous_coco.config
mask_rcnn_in

In [27]:
%cd ~/datalab

import re

#filename = '~/datalab/pretrained_model1/pipeline.config'
filename = '/root/models/research/object_detection/samples/configs/ssd_mobilenet_v1_coco.config'
with open(filename) as f:
  s = f.read()
with open(filename, 'w') as f:
  s = re.sub('PATH_TO_BE_CONFIGURED/model.ckpt', '/root/datalab/pretrained_model1/model.ckpt', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_train.record-\?\?\?\?\?-of-00100', '/root/datalab/tf_train.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_val.record-\?\?\?\?\?-of-00010', '/root/datalab/tf_val.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt', '/root/datalab/label_map.pbtxt', s)
  f.write(s)

/root/datalab


In [28]:
!cat ~/models/research/object_detection/samples/configs/ssd_mobilenet_v1_coco.config

# SSD with Mobilenet v1 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 90
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect

# Train model

Set num_train_steps and num_eval_steps values to change train and eval steps in training process.

# *tensorboard*

In [29]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('tensorboard --logdir /root/datalab --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-05-14 19:27:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.173.6.94, 54.174.228.92, 52.203.66.95, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.173.6.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16529980 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.76M  19.5MB/s    in 0.8s    

2019-05-14 19:28:00 (19.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16529980/16529980]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://6a5424d1.ngrok.io


# train the model

In [0]:
%cd ~/datalab
!python ~/models/research/object_detection/model_main.py \
    --pipeline_config_path=/root/models/research/object_detection/samples/configs/ssd_mobilenet_v1_coco.config \
    --model_dir=/root/datalab/trained \
    --alsologtostderr \
    --num_train_steps=15000 \
    --num_eval_steps=500


/root/datalab

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Use `tf.data.Dataset.batch(..., drop_remainder=True)`.
2019-05-14 19:28:25.881639: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-05-14 19:28:25.881847: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0xc655340 executing computations on platform Host. Device

# Export trained model
Export trained model with highest step number in filename.

In [0]:
%cd ~/datalab/pretrained_model1
!ls
%cd ~/datalab/trained
!ls

In [0]:
%cd ~/datalab

#!mv model.ckpt model.ckpt.meta
#!ls
%cd ~/datalab
lst = os.listdir('trained')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
last_model = sorted(lf)[-1].replace('.meta', '')
print(last_model);
!python ~/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/root/models/research/object_detection/samples/configs/ssd_mobilenet_v1_coco.config \
    --output_directory=/root/datalab/fine_tuned_model \
    --trained_checkpoint_prefix=/root/datalab/trained/$last_model

In [0]:
%cd ~/datalab/fine_tuned_model 
!ls
from google.colab import files
files.download('frozen_inference_graph.pb')
